In [1]:
import pandas as pd 
import numpy as np
import csv

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
from tabulate import tabulate 

In [2]:
## load data sets
train_pd = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/Without Minor Offenses/train_recid_M.csv")
test_pd = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward//data/Without Minor Offenses/test_recid_M.csv")

## get rid of the record with 'p_age_first_offense' == 0
train_pd = train_pd.drop(['person_id', 'screening_date'], axis=1)
test_pd = test_pd.drop(['person_id', 'screening_date'], axis=1)
test_pd = test_pd[test_pd['p_age_first_offense'] != 0]

## split train and test
x_train, y_train = train_pd.values[:, :-1], train_pd.values[:, -1]
x_test, y_test = test_pd.values[:, :-1], test_pd.values[:, -1]

#### convert variable types

In [3]:
variables = ['sex', 'current_violent', 'current_violent20', 'six_month', 'one_year', 'three_year', 'five_year', 'recid_M']
for i in variables:
    train_pd[i] = train_pd[i].astype('category')
    test_pd[i] = test_pd[i].astype('category')

### Random Forest

In [4]:
from sklearn.ensemble import RandomForestClassifier

### Cross Validation

In [5]:
def crossvalidation(X, Y, nfold, depth, N, min_samples_split=2, min_impurity_decrease=0, seed=816):

    """
    
    @parameters:
    - xtrain: training set -- features
    - ytrain: training set -- response variable
    - nfold: n-folds cross validation
    - depth: max split depth
    - N: number of trees
    - min_samples_splits: the minimum number of samples required to split an internal node
    - min_impurity_decrease: a node will be split if that split induced a decrease of the impurity greater
                             than or equal to this value
    - seed: random state
    
    """
    
    ## nfolds cross-validation set up
    #cv = KFold(n_splits=nfold, random_state=seed, shuffle=True)
    cv = StratifiedKFold(n_splits=nfold, random_state=seed, shuffle=True)
    
    ## classifier
    classifier = RandomForestClassifier(max_depth=depth,
                                        n_estimators=N, 
                                        min_impurity_decrease=min_impurity_decrease,
                                        min_samples_split= min_samples_split,
                                        bootstrap=True, 
                                        random_state=seed)
    train_acc, test_acc = [], []
    train_auc, test_auc = [], []

    i = 0
    for train, test in cv.split(X, Y):
    
        ## data & classifier
        X_train, Y_train = X[train], Y[train]
        X_test, Y_test = X[test], Y[test]
        fit_model = classifier.fit(X_train, Y_train)
    
        ## accuracy & probability
        train_acc.append(fit_model.score(X_train, Y_train))
        test_acc.append(fit_model.score(X_test, Y_test))
    
        train_prob = fit_model.predict_proba(X_train)[:,1]
        test_prob = fit_model.predict_proba(X_test)[:,1]
    
        ## compute ROC curve and AUC
    
        train_fpr, train_tpr, train_thresholds = roc_curve(Y_train, train_prob)
        test_fpr, test_tpr, test_thresholds = roc_curve(Y_test, test_prob)
        train_roc_auc = auc(train_fpr, train_tpr)
        test_roc_auc = auc(test_fpr, test_tpr)
    
        train_auc.append(train_roc_auc)
        test_auc.append(test_roc_auc)
        i += 1
    
    return train_acc, test_acc, train_auc, test_auc

### Tune Parameters 
-- To prevent overfitting and get as good performance as possible.

-- criteria: difference between the avg. train accuracy and test accuracy and the difference between avg. train auc and avg. test auc are both smaller than 3%.

In [6]:
Depth = [1,2,3,4,5]
num_estimator = [30, 40, 50, 60, 70, 80, 90, 100]
min_impurity_decrease = [0.007, 0.008, 0.009, 0.011]

In [7]:
results = []

for d in Depth:
    for n in num_estimator:
        for i in min_impurity_decrease:
            train_acc, test_acc, train_auc, test_auc = crossvalidation(x_train, y_train, 5, depth=d, N=n, min_impurity_decrease=i)
            auc_diff = np.mean(train_auc) - np.mean(test_auc)
            results.append([d, n, i, np.mean(test_auc), auc_diff])

In [11]:
table = pd.DataFrame(results, columns=['Depth', 'Number of Estimators', 'Min Impurity Decrease', 'Validation AUC', 'AUC Diff'])

In [13]:
table = table[table['AUC Diff'] <= 0.026]

In [14]:
table.sort_values(by = ['Validation AUC', 'AUC Diff'], axis=0, ascending = [False, True])

,Depth,Number of Estimators,Min Impurity Decrease,Validation AUC,AUC Diff
95,3,100,0.011,0.616052,0.022795
127,4,100,0.011,0.616052,0.022795
159,5,100,0.011,0.616052,0.022795
63,2,100,0.011,0.615887,0.021128
21,1,80,0.008,0.614882,0.021886
91,3,90,0.011,0.614829,0.023815
123,4,90,0.011,0.614829,0.023815
155,5,90,0.011,0.614829,0.023815
59,2,90,0.011,0.614482,0.022152
17,1,70,0.008,0.614074,0.023355


### Best Parameters:
- depth: 3 / estimators: 100 / impurity decrease: 0.011

In [15]:
train_acc, test_acc, train_auc, test_auc = crossvalidation(x_train, y_train, 5, depth=3, N=100, min_impurity_decrease=0.011)

In [16]:
np.mean(train_acc), np.mean(test_acc), np.mean(train_auc), np.mean(test_auc)

(0.7268075400752256, 0.726809314797451, 0.6388476857075507, 0.6160524943994281)

#### Heldout Test Set
-- use 0.5 as threshold

In [17]:
rf = RandomForestClassifier(max_depth=3, n_estimators=100, min_impurity_decrease=0.011, bootstrap=True, random_state=816).fit(x_train, y_train)
heldout_test_acc = rf.score(x_test, y_test)
heldout_test_acc

0.728494623655914

In [18]:
prob = rf.predict_proba(x_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, prob)
heldout_test_auc = auc(fpr, tpr)
heldout_test_auc

0.6031200906068466

-- use optimal threshold

#optimal_index = np.argmin(np.sqrt(np.square(1-tpr) + np.square(fpr)))
optimal_index = np.argmax(abs(tpr - fpr))
optimal_threshold = thresholds[optimal_index]
optimal_threshold

prediction = rf.predict_proba(x_test)[:,1]
heldout_test_acc = np.mean((prediction > optimal_threshold) == y_test)
heldout_test_acc

#### Log Results

In [19]:
#log model results to the model performance folder, as per standards
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\broward\\broward models\\Baseline Model Results\\Without Minor Offenses\\Misdemeanor\\"

train_auc_mean, train_auc_std = np.mean(train_auc), np.std(train_auc)
test_auc_mean, test_auc_std = np.mean(test_auc), np.std(test_auc)
                   
results = ["RF", train_auc_mean, train_auc_std, test_auc_mean, test_auc_std, heldout_test_auc, heldout_test_acc ]

with open(path + 'Misdemeanor Summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerow(results)